In [1]:
import pandas as pd

# Đọc tập dữ liệu CSV
df = pd.read_csv('Data_Clean.csv')


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1368 entries, 0 to 1367
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Tên SP        1367 non-null   object 
 1   Nhà sản xuất  1368 non-null   object 
 2   Model         1368 non-null   object 
 3   Kết nối       1368 non-null   object 
 4   Kích thước    1368 non-null   float64
 5   Loại switch   1368 non-null   object 
 6   Cân nặng      1368 non-null   float64
 7   Giá(đ)        1368 non-null   int64  
dtypes: float64(2), int64(1), object(5)
memory usage: 85.6+ KB
